In [ ]:
# %%
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# %%
# Definición de la función de activación ReLU y su derivada
def ReLU(x):
    return np.maximum(0, x)

def ReLU_derivative(x):
    return np.where(x > 0, 1, 0)

# Inicialización de la red neuronal
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = {
        'hidden_layer_1': {
            'weights': np.random.randn(n_inputs, n_hidden),
            'biases': np.zeros(n_hidden)
        },
        'hidden_layer_2': {
            'weights': np.random.randn(n_hidden, n_hidden),
            'biases': np.zeros(n_hidden)
        },
        'output_layer': {
            'weights': np.random.randn(n_hidden, n_outputs),
            'biases': np.zeros(n_outputs)
        }
    }
    return network

# Propagación hacia adelante
def forward(network, inputs):
    hidden_inputs_1 = np.dot(inputs, network['hidden_layer_1']['weights']) + network['hidden_layer_1']['biases']
    hidden_outputs_1 = ReLU(hidden_inputs_1)
    
    hidden_inputs_2 = np.dot(hidden_outputs_1, network['hidden_layer_2']['weights']) + network['hidden_layer_2']['biases']
    hidden_outputs_2 = ReLU(hidden_inputs_2)
    
    final_inputs = np.dot(hidden_outputs_2, network['output_layer']['weights']) + network['output_layer']['biases']
    final_outputs = ReLU(final_inputs)
    
    return final_outputs, hidden_outputs_1, hidden_outputs_2

# Retropropagación
def backward(network, inputs, outputs, expected_outputs, hidden_outputs_1, hidden_outputs_2):
    expected_outputs = np.array(expected_outputs, ndmin=2)
    output_errors = expected_outputs - outputs
    output_delta = output_errors * ReLU_derivative(outputs)
    
    hidden_2_errors = np.dot(output_delta, network['output_layer']['weights'].T)
    hidden_2_delta = hidden_2_errors * ReLU_derivative(hidden_outputs_2)
    
    hidden_1_errors = np.dot(hidden_2_delta, network['hidden_layer_2']['weights'].T)
    hidden_1_delta = hidden_1_errors * ReLU_derivative(hidden_outputs_1)
    
    l_rate = 0.1
    network['output_layer']['weights']   += l_rate * np.dot(hidden_outputs_2.T, output_delta)
    network['hidden_layer_2']['weights'] += l_rate * np.dot(hidden_outputs_1.T, hidden_2_delta)
    network['hidden_layer_1']['weights'] += l_rate * np.dot(inputs.T, hidden_1_delta)
    
    network['output_layer']['biases']    += l_rate * np.sum(output_delta, axis=0)
    network['hidden_layer_2']['biases']  += l_rate * np.sum(hidden_2_delta, axis=0)
    network['hidden_layer_1']['biases']  += l_rate * np.sum(hidden_1_delta, axis=0)


# Entrenamiento de la red neuronal
def train(network, train_data, n_outputs=1):
    n_epochs = 100
    errors = []
    for _ in range(n_epochs):
        total_error = 0
        for row in train_data:
            inputs = np.array(row[0], ndmin=2)
            outputs, h1, h2 = forward(network, inputs)
            expected_outputs = np.array(row[1], ndmin=2)
            total_error += np.sum((expected_outputs - outputs) ** 2)
            backward(network, inputs, outputs, expected_outputs, h1, h2)
        errors.append(total_error)
    return errors

# Visualización de los resultados del entrenamiento
def plot_errors(errors):
    plt.plot(errors)
    plt.xlabel('Época')
    plt.ylabel('Error Total')
    plt.title('Error a lo largo del entrenamiento')
    plt.show()

In [ ]:
# Código principal
# Carga y preparación de datos
df = pd.read_csv('/home/jd/Documentos/CODIGO/Machine-Learning/tutorials/ai-from-scratch/resources/btc.csv')
train_dataset = [(row[['open', 'high', 'low']].tolist(), [row['close']]) for index, row in df.iterrows()]

# Normalización de los datos (ejemplo simple)
max_value = df[['open', 'high', 'low', 'close']].max().max()
normalized_dataset = [(np.array(row[0]) / max_value, [row[1][0] / max_value]) for row in train_dataset]

In [ ]:
network = initialize_network(n_inputs=3, n_hidden=3, n_outputs=1)
errors = train(network, normalized_dataset)
plot_errors(errors)

In [ ]:
# Inferencia utilizando la red neuronal entrenada
def predict(network, input_data):
    outputs, _, _ = forward(network, np.array(input_data, ndmin=2))
    return outputs

In [ ]:
# Inferencia
test_input = np.array([40000, 41000, 39000]) / max_value
predicted_close_price = predict(network, test_input) * max_value
print(f"El precio de cierre predicho es: {predicted_close_price}")